# Feature Engineering

### Get data from DB

In [1]:
import pandas as pd
import numpy as np
import sqlite3

pd.set_option("display.max_columns", None)

conn = sqlite3.connect("../data/ufc.db", detect_types=sqlite3.PARSE_DECLTYPES)
query = """
        SELECT
            a.*,
            b.HEIGHT_INCHES AS RED_HEIGHT_INCHES,
            b.REACH_INCHES AS RED_REACH_INCHES,
            b.FIGHTING_STANCE AS RED_FIGHTING_STANCE,
            b.DATE_OF_BIRTH AS RED_DATE_OF_BIRTH,
            c.HEIGHT_INCHES AS BLUE_HEIGHT_INCHES,
            c.REACH_INCHES AS BLUE_REACH_INCHES,
            c.FIGHTING_STANCE AS BLUE_FIGHTING_STANCE,
            c.DATE_OF_BIRTH AS BLUE_DATE_OF_BIRTH
        FROM
            BOUTS_OVERALL a
        LEFT JOIN
            FIGHTERS b
        ON
            a.RED_FIGHTER_ID = b.FIGHTER_ID
        LEFT JOIN
            FIGHTERS c
        ON
            a.BLUE_FIGHTER_ID = c.FIGHTER_ID
        """

df = pd.read_sql_query(query, conn)

### Create a stacked dataset so that features can be engineered by fighter

In [2]:
def get_weight_class(bout_type):
    if "Women's Strawweight" in bout_type:
        return "Women's Strawweight"
    elif "Women's Flyweight" in bout_type:
        return "Women's Flyweight"
    elif "Women's Bantamweight" in bout_type:
        return "Women's Bantamweight"
    elif "Women's Featherweight" in bout_type:
        return "Women's Featherweight"
    elif "Light Heavyweight" in bout_type:
        return "Light Heavyweight"
    elif "Heavyweight" in bout_type:
        return "Heavyweight"
    elif "Middleweight" in bout_type:
        return "Middleweight"
    elif "Welterweight" in bout_type:
        return "Welterweight"
    elif "Lightweight" in bout_type:
        return "Lightweight"
    elif "Featherweight" in bout_type:
        return "Featherweight"
    elif "Bantamweight" in bout_type:
        return "Bantamweight"
    elif "Flyweight" in bout_type:
        return "Flyweight"
    elif "Catch Weight" in bout_type:
        return "Catch Weight"
    elif "Open Weight" in bout_type:
        return "Open Weight"
    
def is_female(bout_type):
    if "Women's" in bout_type:
        return 1
    return 0

def is_title_bout(bout_type):
    if any(x in bout_type for x in ["Tournament Title Bout", "Ultimate Fighter", "TUF"]):
        return 0
    elif "Title Bout" in bout_type:
        return 1
    return 0

def is_undisputed_title_bout(bout_type):
    if any(x in bout_type for x in ["Tournament Title Bout", "Ultimate Fighter", "TUF"]):
        return 0
    elif "Title Bout" in bout_type:
        if "Interim" in bout_type:
            return 0
        return 1
    return 0

In [3]:
temp = df.copy().rename_axis("TEMP").reset_index()
common = [
    "TEMP",
    "BOUT_ID",
    "DATE",
    # "LOCATION",
    "BOUT_TYPE",
    "OUTCOME_METHOD",
    "END_ROUND",
    "BOUT_TIME_FORMAT",
    "TOTAL_TIME_MINUTES",
]
red_cols = [col for col in temp.columns if col.startswith("RED_")]
blue_cols = [col for col in temp.columns if col.startswith("BLUE_")]

reds = temp[common + red_cols + blue_cols]
reds = reds.rename(
    columns=lambda x: x.replace("RED_", "f_")
    if x.startswith("RED_")
    else x.replace("BLUE_", "o_")
    if x.startswith("BLUE_")
    else x
)
reds["TEMP2"] = 1
blues = temp[common + blue_cols + red_cols]
blues = blues.rename(
    columns=lambda x: x.replace("BLUE_", "f_")
    if x.startswith("BLUE_")
    else x.replace("RED_", "o_")
    if x.startswith("RED_")
    else x
)
blues["TEMP2"] = 2

feature_df = (
    pd.concat([reds, blues])
    .sort_values(["TEMP", "TEMP2"])
    .drop(["TEMP", "TEMP2", "f_FIGHTER_NAME", "o_FIGHTER_NAME", "o_DATE_OF_BIRTH", "o_FIGHTER_ID"], axis=1)
)
feature_df["DATE"] = pd.to_datetime(feature_df["DATE"])
feature_df["f_DATE_OF_BIRTH"] = pd.to_datetime(feature_df["f_DATE_OF_BIRTH"])
feature_df = feature_df.drop("o_OUTCOME", axis=1)

# Bout type feature engineering
feature_df["BOUT_WEIGHT_CLASS"] = feature_df["BOUT_TYPE"].apply(lambda x: get_weight_class(x))
feature_df["BOUT_IS_FEMALE"] = feature_df["BOUT_TYPE"].apply(lambda x: is_female(x))
feature_df["BOUT_IS_TITLE_BOUT"] = feature_df["BOUT_TYPE"].apply(lambda x: is_title_bout(x))
feature_df["BOUT_IS_UNDISPUTED_TITLE_BOUT"] = feature_df["BOUT_TYPE"].apply(lambda x: is_undisputed_title_bout(x))
feature_df = feature_df.drop("BOUT_TYPE", axis=1)

# Outcome and method feature engineering
feature_df["f_WIN"] = feature_df["f_OUTCOME"].apply(lambda x: 1 if x == "W" else 0)
feature_df["f_LOSS"] = feature_df["f_OUTCOME"].apply(lambda x: 1 if x == "L" else 0)
feature_df["f_DRAW"] = feature_df["f_OUTCOME"].apply(lambda x: 1 if x == "D" else 0)
feature_df["f_NO_CONTEST"] = feature_df["f_OUTCOME"].apply(lambda x: 1 if x == "NC" else 0)
feature_df["f_WIN_BY_KO/TKO"] = feature_df.apply(lambda x: 1 if (x["f_OUTCOME"] == "W" and x["OUTCOME_METHOD"] == "KO/TKO") else 0, axis=1)
feature_df["f_WIN_BY_SUBMISSION"] = feature_df.apply(lambda x: 1 if (x["f_OUTCOME"] == "W" and x["OUTCOME_METHOD"] == "Submission") else 0, axis=1)
feature_df["f_WIN_BY_DECISION"] = feature_df.apply(lambda x: 1 if (x["f_OUTCOME"] == "W" and "Decision" in x["OUTCOME_METHOD"]) else 0, axis=1)
feature_df["f_LOSS_BY_KO/TKO"] = feature_df.apply(lambda x: 1 if (x["f_OUTCOME"] == "L" and x["OUTCOME_METHOD"] == "KO/TKO") else 0, axis=1)
feature_df["f_LOSS_BY_SUBMISSION"] = feature_df.apply(lambda x: 1 if (x["f_OUTCOME"] == "L" and x["OUTCOME_METHOD"] == "Submission") else 0, axis=1)
feature_df["f_LOSS_BY_DECISION"] = feature_df.apply(lambda x: 1 if (x["f_OUTCOME"] == "L" and "Decision" in x["OUTCOME_METHOD"]) else 0, axis=1)
feature_df = feature_df.drop(["f_OUTCOME", "OUTCOME_METHOD"], axis=1)

# End round and time format stuff
feature_df["f_WIN_FIRST_ROUND_FINISH"] = feature_df.apply(lambda x: 1 if (x["f_WIN"] == 1 and x["END_ROUND"] == 1) else 0, axis=1)
feature_df["BOUT_IS_FIVE_ROUNDS"] = feature_df["BOUT_TIME_FORMAT"].apply(lambda x: 1 if x == "5 Rnd (5-5-5-5-5)" else 0)
feature_df = feature_df.drop(["END_ROUND", "BOUT_TIME_FORMAT"], axis=1)

feature_df

,BOUT_ID,DATE,TOTAL_TIME_MINUTES,f_FIGHTER_ID,f_KNOCKDOWNS,f_TOTAL_STRIKES_LANDED,f_TOTAL_STRIKES_ATTEMPTED,f_TAKEDOWNS_LANDED,f_TAKEDOWNS_ATTEMPTED,f_SUBMISSION_ATTEMPTS,f_REVERSALS,f_CONTROL_TIME_MINUTES,f_SIGNIFICANT_STRIKES_LANDED,f_SIGNIFICANT_STRIKES_ATTEMPTED,f_SIGNIFICANT_STRIKES_HEAD_LANDED,f_SIGNIFICANT_STRIKES_HEAD_ATTEMPTED,f_SIGNIFICANT_STRIKES_BODY_LANDED,f_SIGNIFICANT_STRIKES_BODY_ATTEMPTED,f_SIGNIFICANT_STRIKES_LEG_LANDED,f_SIGNIFICANT_STRIKES_LEG_ATTEMPTED,f_SIGNIFICANT_STRIKES_DISTANCE_LANDED,f_SIGNIFICANT_STRIKES_DISTANCE_ATTEMPTED,f_SIGNIFICANT_STRIKES_CLINCH_LANDED,f_SIGNIFICANT_STRIKES_CLINCH_ATTEMPTED,f_SIGNIFICANT_STRIKES_GROUND_LANDED,f_SIGNIFICANT_STRIKES_GROUND_ATTEMPTED,f_HEIGHT_INCHES,f_REACH_INCHES,f_FIGHTING_STANCE,f_DATE_OF_BIRTH,o_KNOCKDOWNS,o_TOTAL_STRIKES_LANDED,o_TOTAL_STRIKES_ATTEMPTED,o_TAKEDOWNS_LANDED,o_TAKEDOWNS_ATTEMPTED,o_SUBMISSION_ATTEMPTS,o_REVERSALS,o_CONTROL_TIME_MINUTES,o_SIGNIFICANT_STRIKES_LANDED,o_SIGNIFICANT_STRIKES_ATTEMPTED,o_SIGNIFICANT_STRIKES_HEAD_LANDED,o_SIGNIFICANT_STRIKES_HEAD_ATTEMPTED,o_SIGNIFICANT_STRIKES_BODY_LANDED,o_SIGNIFICANT_STRIKES_BODY_ATTEMPTED,o_SIGNIFICANT_STRIKES_LEG_LANDED,o_SIGNIFICANT_STRIKES_LEG_ATTEMPTED,o_SIGNIFICANT_STRIKES_DISTANCE_LANDED,o_SIGNIFICANT_STRIKES_DISTANCE_ATTEMPTED,o_SIGNIFICANT_STRIKES_CLINCH_LANDED,o_SIGNIFICANT_STRIKES_CLINCH_ATTEMPTED,o_SIGNIFICANT_STRIKES_GROUND_LANDED,o_SIGNIFICANT_STRIKES_GROUND_ATTEMPTED,o_HEIGHT_INCHES,o_REACH_INCHES,o_FIGHTING_STANCE,BOUT_WEIGHT_CLASS,BOUT_IS_FEMALE,BOUT_IS_TITLE_BOUT,BOUT_IS_UNDISPUTED_TITLE_BOUT,f_WIN,f_LOSS,f_DRAW,f_NO_CONTEST,f_WIN_BY_KO/TKO,f_WIN_BY_SUBMISSION,f_WIN_BY_DECISION,f_LOSS_BY_KO/TKO,f_LOSS_BY_SUBMISSION,f_LOSS_BY_DECISION,f_WIN_FIRST_ROUND_FINISH,BOUT_IS_FIVE_ROUNDS
0,567a09fd200cfa05,1993-11-12,0.433333,279093302a6f44b3,0.0,3.0,5.0,0.0,0.0,0.0,0.0,NaN,3.0,5.0,3.0,5.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0,0.0,2.0,2.0,77.0,NaN,Orthodox,1959-03-30,0.0,0.0,1.0,0.0,1.0,0.0,0.0,NaN,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,72.0,NaN,Orthodox,Open Weight,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0
0,567a09fd200cfa05,1993-11-12,0.433333,96eff1a628adcc7f,0.0,0.0,1.0,0.0,1.0,0.0,0.0,NaN,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,72.0,NaN,Orthodox,1969-06-14,0.0,3.0,5.0,0.0,0.0,0.0,0.0,NaN,3.0,5.0,3.0,5.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0,0.0,2.0,2.0,77.0,NaN,Orthodox,Open Weight,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
1,2d2bbc86e941e05c,1993-11-12,4.333333,598a58db87b890ee,2.0,38.0,53.0,0.0,0.0,0.0,0.0,NaN,15.0,27.0,12.0,23.0,3.0,4.0,0.0,0.0,4.0,10.0,4.0,9.0,7.0,8.0,76.0,NaN,Orthodox,NaT,0.0,13.0,29.0,0.0,0.0,0.0,0.0,NaN,12.0,28.0,7.0,19.0,3.0,6.0,2.0,3.0,0.0,7.0,10.0,19.0,2.0,2.0,77.0,NaN,Orthodox,Open Weight,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0
1,2d2bbc86e941e05c,1993-11-12,4.333333,d3711d3784b76255,0.0,13.0,29.0,0.0,0.0,0.0,0.0,NaN,12.0,28.0,7.0,19.0,3.0,6.0,2.0,3.0,0.0,7.0,10.0,19.0,2.0,2.0,77.0,NaN,Orthodox,NaT,2.0,38.0,53.0,0.0,0.0,0.0,0.0,NaN,15.0,27.0,12.0,23.0,3.0,4.0,0.0,0.0,4.0,10.0,4.0,9.0,7.0,8.0,76.0,NaN,Orthodox,Open Weight,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
2,cecdc0da584274b9,1993-11-12,2.300000,429e7d3725852ce9,0.0,4.0,7.0,1.0,1.0,0.0,0.0,NaN,0.0,3.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,3.0,0.0,0.0,0.0,0.0,73.0,NaN,Southpaw,1966-12-12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,73.0,NaN,Orthodox,Open Weight,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7286,c57a4810c932b296,2023-08-12,2.666667,d0aaedcf7ccd0b45,0.0,14.0,28.0,0.0,1.0,0.0,0.0,0.066667,8.0,21.0,6.0,19.0,2.0,2.0,0.0,0.0,7.0,19.0,1.0,2.0,0.0,0.0,75.0,76.0,Orthodox,1989-09-25,1.0,16.0,24.0,0.0,0.0,0.0,0.0,0.516667,16.0,24.0,12.0,19.0,3.0,4.0,1.0,1.0,11.0,17.0,3.0,4.0,2.0,3.0,73.0,76.0,Southpaw,Light Heavyweight,0,0,0,0,1,0,0,0

In [8]:
cumulative_features = pd.DataFrame()
cumulative_features["f_TOTAL_TIME_MINUTES_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["TOTAL_TIME_MINUTES"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["f_KNOCKDOWNS_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["f_KNOCKDOWNS"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["f_TOTAL_STRIKES_LANDED_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["f_TOTAL_STRIKES_LANDED"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["f_TOTAL_STRIKES_ATTEMPTED_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["f_TOTAL_STRIKES_ATTEMPTED"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["f_TAKEDOWNS_LANDED_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["f_TAKEDOWNS_LANDED"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["f_TAKEDOWNS_ATTEMPTED_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["f_TAKEDOWNS_ATTEMPTED"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["f_SUBMISSION_ATTEMPTS_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["f_SUBMISSION_ATTEMPTS"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["f_REVERSALS_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["f_REVERSALS"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["f_CONTROL_TIME_MINUTES_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["f_CONTROL_TIME_MINUTES"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["f_SIGNIFICANT_STRIKES_LANDED_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["f_SIGNIFICANT_STRIKES_LANDED"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["f_SIGNIFICANT_STRIKES_ATTEMPTED_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["f_SIGNIFICANT_STRIKES_ATTEMPTED"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["f_SIGNIFICANT_STRIKES_HEAD_LANDED_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["f_SIGNIFICANT_STRIKES_HEAD_LANDED"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["f_SIGNIFICANT_STRIKES_HEAD_ATTEMPTED_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["f_SIGNIFICANT_STRIKES_HEAD_ATTEMPTED"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["f_SIGNIFICANT_STRIKES_BODY_LANDED_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["f_SIGNIFICANT_STRIKES_BODY_LANDED"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["f_SIGNIFICANT_STRIKES_BODY_ATTEMPTED_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["f_SIGNIFICANT_STRIKES_BODY_ATTEMPTED"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["f_SIGNIFICANT_STRIKES_LEG_LANDED_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["f_SIGNIFICANT_STRIKES_LEG_LANDED"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["f_SIGNIFICANT_STRIKES_LEG_ATTEMPTED_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["f_SIGNIFICANT_STRIKES_LEG_ATTEMPTED"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["f_SIGNIFICANT_STRIKES_DISTANCE_LANDED_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["f_SIGNIFICANT_STRIKES_DISTANCE_LANDED"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["f_SIGNIFICANT_STRIKES_DISTANCE_ATTEMPTED_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["f_SIGNIFICANT_STRIKES_DISTANCE_ATTEMPTED"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["f_SIGNIFICANT_STRIKES_CLINCH_LANDED_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["f_SIGNIFICANT_STRIKES_CLINCH_LANDED"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["f_SIGNIFICANT_STRIKES_CLINCH_ATTEMPTED_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["f_SIGNIFICANT_STRIKES_CLINCH_ATTEMPTED"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["f_SIGNIFICANT_STRIKES_GROUND_LANDED_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["f_SIGNIFICANT_STRIKES_GROUND_LANDED"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["f_SIGNIFICANT_STRIKES_GROUND_ATTEMPTED_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["f_SIGNIFICANT_STRIKES_GROUND_ATTEMPTED"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["o_KNOCKDOWNS_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["o_KNOCKDOWNS"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["o_TOTAL_STRIKES_LANDED_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["o_TOTAL_STRIKES_LANDED"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["o_TOTAL_STRIKES_ATTEMPTED_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["o_TOTAL_STRIKES_ATTEMPTED"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["o_TAKEDOWNS_LANDED_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["o_TAKEDOWNS_LANDED"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["o_TAKEDOWNS_ATTEMPTED_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["o_TAKEDOWNS_ATTEMPTED"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["o_SUBMISSION_ATTEMPTS_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["o_SUBMISSION_ATTEMPTS"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["o_REVERSALS_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["o_REVERSALS"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["o_CONTROL_TIME_MINUTES_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["o_CONTROL_TIME_MINUTES"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["o_SIGNIFICANT_STRIKES_LANDED_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["o_SIGNIFICANT_STRIKES_LANDED"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["o_SIGNIFICANT_STRIKES_ATTEMPTED_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["o_SIGNIFICANT_STRIKES_ATTEMPTED"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["o_SIGNIFICANT_STRIKES_HEAD_LANDED_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["o_SIGNIFICANT_STRIKES_HEAD_LANDED"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["o_SIGNIFICANT_STRIKES_HEAD_ATTEMPTED_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["o_SIGNIFICANT_STRIKES_HEAD_ATTEMPTED"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["o_SIGNIFICANT_STRIKES_BODY_LANDED_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["o_SIGNIFICANT_STRIKES_BODY_LANDED"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["o_SIGNIFICANT_STRIKES_BODY_ATTEMPTED_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["o_SIGNIFICANT_STRIKES_BODY_ATTEMPTED"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["o_SIGNIFICANT_STRIKES_LEG_LANDED_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["o_SIGNIFICANT_STRIKES_LEG_LANDED"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["o_SIGNIFICANT_STRIKES_LEG_ATTEMPTED_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["o_SIGNIFICANT_STRIKES_LEG_ATTEMPTED"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["o_SIGNIFICANT_STRIKES_DISTANCE_LANDED_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["o_SIGNIFICANT_STRIKES_DISTANCE_LANDED"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["o_SIGNIFICANT_STRIKES_DISTANCE_ATTEMPTED_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["o_SIGNIFICANT_STRIKES_DISTANCE_ATTEMPTED"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["o_SIGNIFICANT_STRIKES_CLINCH_LANDED_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["o_SIGNIFICANT_STRIKES_CLINCH_LANDED"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["o_SIGNIFICANT_STRIKES_CLINCH_ATTEMPTED_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["o_SIGNIFICANT_STRIKES_CLINCH_ATTEMPTED"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["o_SIGNIFICANT_STRIKES_GROUND_LANDED_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["o_SIGNIFICANT_STRIKES_GROUND_LANDED"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["o_SIGNIFICANT_STRIKES_GROUND_ATTEMPTED_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["o_SIGNIFICANT_STRIKES_GROUND_ATTEMPTED"].transform(lambda x: x.cumsum().shift(1))

cumulative_features["f_TITLE_BOUTS_FOUGHT"] = feature_df.groupby("f_FIGHTER_ID")["BOUT_IS_TITLE_BOUT"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["f_UNDISPUTED_TITLE_BOUTS_FOUGHT"] = feature_df.groupby("f_FIGHTER_ID")["BOUT_IS_UNDISPUTED_TITLE_BOUT"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["f_FIVE_ROUND_BOUTS_FOUGHT"] = feature_df.groupby("f_FIGHTER_ID")["BOUT_IS_FIVE_ROUNDS"].transform(lambda x: x.cumsum().shift(1))

cumulative_features["f_WINS_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["f_WIN"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["f_LOSSES_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["f_LOSS"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["f_DRAWS_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["f_DRAW"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["f_NO_CONTESTS_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["f_NO_CONTEST"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["f_TOTAL_BOUTS_FOUGHT"] = cumulative_features["f_WINS_CUMULATIVE"] + cumulative_features["f_LOSSES_CUMULATIVE"] + cumulative_features["f_DRAWS_CUMULATIVE"] + cumulative_features["f_NO_CONTESTS_CUMULATIVE"]
cumulative_features["f_WINS_BY_KO/TKO_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["f_WIN_BY_KO/TKO"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["f_WINS_BY_SUBMISSION_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["f_WIN_BY_SUBMISSION"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["f_WINS_BY_DECISION_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["f_WIN_BY_DECISION"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["f_WINS_FIRST_ROUND_FINISH_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["f_WIN_FIRST_ROUND_FINISH"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["f_LOSSES_BY_KO/TKO_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["f_LOSS_BY_KO/TKO"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["f_LOSSES_BY_SUBMISSION_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["f_LOSS_BY_SUBMISSION"].transform(lambda x: x.cumsum().shift(1))
cumulative_features["f_LOSSES_BY_DECISION_CUMULATIVE"] = feature_df.groupby("f_FIGHTER_ID")["f_LOSS_BY_DECISION"].transform(lambda x: x.cumsum().shift(1))

cumulative_features

,f_TOTAL_TIME_MINUTES_CUMULATIVE,f_KNOCKDOWNS_CUMULATIVE,f_TOTAL_STRIKES_LANDED_CUMULATIVE,f_TOTAL_STRIKES_ATTEMPTED_CUMULATIVE,f_TAKEDOWNS_LANDED_CUMULATIVE,f_TAKEDOWNS_ATTEMPTED_CUMULATIVE,f_SUBMISSION_ATTEMPTS_CUMULATIVE,f_REVERSALS_CUMULATIVE,f_CONTROL_TIME_MINUTES_CUMULATIVE,f_SIGNIFICANT_STRIKES_LANDED_CUMULATIVE,f_SIGNIFICANT_STRIKES_ATTEMPTED_CUMULATIVE,f_SIGNIFICANT_STRIKES_HEAD_LANDED_CUMULATIVE,f_SIGNIFICANT_STRIKES_HEAD_ATTEMPTED_CUMULATIVE,f_SIGNIFICANT_STRIKES_BODY_LANDED_CUMULATIVE,f_SIGNIFICANT_STRIKES_BODY_ATTEMPTED_CUMULATIVE,f_SIGNIFICANT_STRIKES_LEG_LANDED_CUMULATIVE,f_SIGNIFICANT_STRIKES_LEG_ATTEMPTED_CUMULATIVE,f_SIGNIFICANT_STRIKES_DISTANCE_LANDED_CUMULATIVE,f_SIGNIFICANT_STRIKES_DISTANCE_ATTEMPTED_CUMULATIVE,f_SIGNIFICANT_STRIKES_CLINCH_LANDED_CUMULATIVE,f_SIGNIFICANT_STRIKES_CLINCH_ATTEMPTED_CUMULATIVE,f_SIGNIFICANT_STRIKES_GROUND_LANDED_CUMULATIVE,f_SIGNIFICANT_STRIKES_GROUND_ATTEMPTED_CUMULATIVE,o_KNOCKDOWNS_CUMULATIVE,o_TOTAL_STRIKES_LANDED_CUMULATIVE,o_TOTAL_STRIKES_ATTEMPTED_CUMULATIVE,o_TAKEDOWNS_LANDED_CUMULATIVE,o_TAKEDOWNS_ATTEMPTED_CUMULATIVE,o_SUBMISSION_ATTEMPTS_CUMULATIVE,o_REVERSALS_CUMULATIVE,o_CONTROL_TIME_MINUTES_CUMULATIVE,o_SIGNIFICANT_STRIKES_LANDED_CUMULATIVE,o_SIGNIFICANT_STRIKES_ATTEMPTED_CUMULATIVE,o_SIGNIFICANT_STRIKES_HEAD_LANDED_CUMULATIVE,o_SIGNIFICANT_STRIKES_HEAD_ATTEMPTED_CUMULATIVE,o_SIGNIFICANT_STRIKES_BODY_LANDED_CUMULATIVE,o_SIGNIFICANT_STRIKES_BODY_ATTEMPTED_CUMULATIVE,o_SIGNIFICANT_STRIKES_LEG_LANDED_CUMULATIVE,o_SIGNIFICANT_STRIKES_LEG_ATTEMPTED_CUMULATIVE,o_SIGNIFICANT_STRIKES_DISTANCE_LANDED_CUMULATIVE,o_SIGNIFICANT_STRIKES_DISTANCE_ATTEMPTED_CUMULATIVE,o_SIGNIFICANT_STRIKES_CLINCH_LANDED_CUMULATIVE,o_SIGNIFICANT_STRIKES_CLINCH_ATTEMPTED_CUMULATIVE,o_SIGNIFICANT_STRIKES_GROUND_LANDED_CUMULATIVE,o_SIGNIFICANT_STRIKES_GROUND_ATTEMPTED_CUMULATIVE,f_TITLE_BOUTS_FOUGHT,f_UNDISPUTED_TITLE_BOUTS_FOUGHT,f_FIVE_ROUND_BOUTS_FOUGHT,f_WINS_CUMULATIVE,f_LOSSES_CUMULATIVE,f_DRAWS_CUMULATIVE,f_NO_CONTESTS_CUMULATIVE,f_TOTAL_BOUTS_FOUGHT,f_WINS_BY_KO/TKO_CUMULATIVE,f_WINS_BY_SUBMISSION_CUMULATIVE,f_WINS_BY_DECISION_CUMULATIVE,f_WINS_FIRST_ROUND_FINISH_CUMULATIVE,f_LOSSES_BY_KO/TKO_CUMULATIVE,f_LOSSES_BY_SUBMISSION_CUMULATIVE,f_LOSSES_BY_DECISION_CUMULATIVE
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7286,22.783333,6.0,154.0,295.0,0.0,0.0,0.0,0.0,2.100000,146.0,287.0,127.0,263.0,10.0,14.0,9.0,10.0,103.0,222.0,12.0,19.0,31.0,46.0,3.0,104.0,243.0,0.0,2.0,0.0,0.0,0.550000,102.0,241.0,75.0,205.0,10.0,19.0,17.0,17.0,86.0,217.0,7.0,11.0,9.0,13.0,0.0,0.0,2.0,4.0,3.0,0.0,0.0,7.0,4.0,0.0,0.0,3.0,3.0,0.0,0.

In [26]:
cumulative_features2 = cumulative_features.copy()
cumulative_features2["f_KNOCKDOWNS_LANDED_PER_15_MINUTES_CUMULATIVE"] = cumulative_features2["f_KNOCKDOWNS_CUMULATIVE"] / cumulative_features2["f_TOTAL_TIME_MINUTES_CUMULATIVE"] * 15
cumulative_features2["f_KNOCKDOWNS_ABSORBED_PER_15_MINUTES_CUMULATIVE"] = cumulative_features2["o_KNOCKDOWNS_CUMULATIVE"] / cumulative_features2["f_TOTAL_TIME_MINUTES_CUMULATIVE"] * 15
cumulative_features2["f_KNOCKDOWNS_LANDED/ABSORBED_PER_15_MINUTES_DIFF_CUMULATIVE"] = cumulative_features2["f_KNOCKDOWNS_LANDED_PER_15_MINUTES_CUMULATIVE"] - cumulative_features2["f_KNOCKDOWNS_ABSORBED_PER_15_MINUTES_CUMULATIVE"]
cumulative_features2["f_KNOCKDOWNS_LANDED/ABSORBED_DIFF_CUMULATIVE"] = cumulative_features2["f_KNOCKDOWNS_CUMULATIVE"] - cumulative_features2["o_KNOCKDOWNS_CUMULATIVE"]

cumulative_features2["f_TOTAL_STRIKES_ACCURACY_PROPORTION_CUMULATIVE"] = cumulative_features2["f_TOTAL_STRIKES_LANDED_CUMULATIVE"] / cumulative_features2["f_TOTAL_STRIKES_ATTEMPTED_CUMULATIVE"]
cumulative_features2["f_TOTAL_STRIKES_DEFENSE_PROPORTION_CUMULATIVE"] = (cumulative_features2["o_TOTAL_STRIKES_ATTEMPTED_CUMULATIVE"] - cumulative_features2["o_TOTAL_STRIKES_LANDED_CUMULATIVE"]) / cumulative_features2["o_TOTAL_STRIKES_ATTEMPTED_CUMULATIVE"]
cumulative_features2["f_TOTAL_STRIKES_ACCURACY/DEFENSE_MULT_CUMULATIVE"] = cumulative_features2["f_TOTAL_STRIKES_ACCURACY_PROPORTION_CUMULATIVE"] * cumulative_features2["f_TOTAL_STRIKES_DEFENSE_PROPORTION_CUMULATIVE"]
cumulative_features2["f_TOTAL_STRIKES_LANDED_PER_MINUTE_CUMULATIVE"] = cumulative_features2["f_TOTAL_STRIKES_LANDED_CUMULATIVE"] / cumulative_features2["f_TOTAL_TIME_MINUTES_CUMULATIVE"]
cumulative_features2["f_TOTAL_STRIKES_ABSORBED_PER_MINUTE_CUMULATIVE"] = cumulative_features2["o_TOTAL_STRIKES_LANDED_CUMULATIVE"] / cumulative_features2["f_TOTAL_TIME_MINUTES_CUMULATIVE"]
cumulative_features2["f_TOTAL_STRIKES_LANDED/ABSORBED_PER_MINUTE_DIFF_CUMULATIVE"] = cumulative_features2["f_TOTAL_STRIKES_LANDED_PER_MINUTE_CUMULATIVE"] - cumulative_features2["f_TOTAL_STRIKES_ABSORBED_PER_MINUTE_CUMULATIVE"]
cumulative_features2["f_TOTAL_STRIKES_LANDED/ABSORBED_DIFF_CUMULATIVE"] = cumulative_features2["f_TOTAL_STRIKES_LANDED_CUMULATIVE"] - cumulative_features2["o_TOTAL_STRIKES_LANDED_CUMULATIVE"]

cumulative_features2["f_TAKEDOWNS_ACCURACY_PROPORTION_CUMULATIVE"] = cumulative_features2["f_TAKEDOWNS_LANDED_CUMULATIVE"] / cumulative_features2["f_TAKEDOWNS_ATTEMPTED_CUMULATIVE"]
cumulative_features2["f_TAKEDOWNS_DEFENSE_PROPORTION_CUMULATIVE"] = (cumulative_features2["o_TAKEDOWNS_ATTEMPTED_CUMULATIVE"] - cumulative_features2["o_TAKEDOWNS_LANDED_CUMULATIVE"]) / cumulative_features2["o_TAKEDOWNS_ATTEMPTED_CUMULATIVE"]
cumulative_features2["f_TAKEDOWNS_ACCURACY/DEFENSE_MULT_CUMULATIVE"] = cumulative_features2["f_TAKEDOWNS_ACCURACY_PROPORTION_CUMULATIVE"] * cumulative_features2["f_TAKEDOWNS_DEFENSE_PROPORTION_CUMULATIVE"]
cumulative_features2["f_TAKEDOWNS_LANDED_PER_15_MINUTES_CUMULATIVE"] = cumulative_features2["f_TAKEDOWNS_LANDED_CUMULATIVE"] / cumulative_features2["f_TOTAL_TIME_MINUTES_CUMULATIVE"] * 15
cumulative_features2["f_TAKEDOWNS_ABSORBED_PER_15_MINUTES_CUMULATIVE"] = cumulative_features2["o_TAKEDOWNS_LANDED_CUMULATIVE"] / cumulative_features2["f_TOTAL_TIME_MINUTES_CUMULATIVE"] * 15
cumulative_features2["f_TAKEDOWNS_LANDED/ABSORBED_PER_15_MINUTES_DIFF_CUMULATIVE"] = cumulative_features2["f_TAKEDOWNS_LANDED_PER_15_MINUTES_CUMULATIVE"] - cumulative_features2["f_TAKEDOWNS_ABSORBED_PER_15_MINUTES_CUMULATIVE"]
cumulative_features2["f_TAKEDOWNS_LANDED/ABSORBED_DIFF_CUMULATIVE"] = cumulative_features2["f_TAKEDOWNS_LANDED_CUMULATIVE"] - cumulative_features2["o_TAKEDOWNS_LANDED_CUMULATIVE"]

cumulative_features2["f_SUBMISSION_ATTEMPTS_ACCURACY_PROPORTION_CUMULATIVE"] = cumulative_features2["f_WINS_BY_SUBMISSION_CUMULATIVE"] / cumulative_features2["f_SUBMISSION_ATTEMPTS_CUMULATIVE"]
cumulative_features2["f_SUBMISSION_ATTEMPTS_DEFENSE_PROPORTION_CUMULATIVE"] = (cumulative_features2["o_SUBMISSION_ATTEMPTS_CUMULATIVE"] - cumulative_features2["f_LOSSES_BY_SUBMISSION_CUMULATIVE"]) / cumulative_features2["o_SUBMISSION_ATTEMPTS_CUMULATIVE"]
cumulative_features2["f_SUBMISSION_ATTEMPTS_ACCURACY/DEFENSE_MULT_CUMULATIVE"] = cumulative_features2["f_SUBMISSION_ATTEMPTS_ACCURACY_PROPORTION_CUMULATIVE"] * cumulative_features2["f_SUBMISSION_ATTEMPTS_DEFENSE_PROPORTION_CUMULATIVE"]
cumulative_features2["f_SUBMISSION_ATTEMPTS_PER_15_MINUTES_CUMULATIVE"] = cumulative_features2["f_SUBMISSION_ATTEMPTS_CUMULATIVE"] / cumulative_features2["f_TOTAL_TIME_MINUTES_CUMULATIVE"] * 15
cumulative_features2["f_SUBMISSION_ATTEMPTS_ABSORBED_PER_15_MINUTES_CUMULATIVE"] = cumulative_features2["o_SUBMISSION_ATTEMPTS_CUMULATIVE"] / cumulative_features2["f_TOTAL_TIME_MINUTES_CUMULATIVE"] * 15
cumulative_features2["f_SUBMISSION_ATTEMPTS/ABSORBED_PER_15_MINUTES_DIFF_CUMULATIVE"] = cumulative_features2["f_SUBMISSION_ATTEMPTS_PER_15_MINUTES_CUMULATIVE"] - cumulative_features2["f_SUBMISSION_ATTEMPTS_ABSORBED_PER_15_MINUTES_CUMULATIVE"]
cumulative_features2["f_SUBMISSION_ATTEMPTS/ABSORBED_DIFF_CUMULATIVE"] = cumulative_features2["f_SUBMISSION_ATTEMPTS_CUMULATIVE"] - cumulative_features2["o_SUBMISSION_ATTEMPTS_CUMULATIVE"]

cumulative_features2["f_REVERSALS_ACCURACY_PROPORTION_CUMULATIVE"] = cumulative_features2["f_REVERSALS_CUMULATIVE"] / cumulative_features2["o_TAKEDOWNS_LANDED_CUMULATIVE"]
cumulative_features2["f_REVERSALS_DEFENSE_PROPORTION_CUMULATIVE"] = (cumulative_features2["f_TAKEDOWNS_LANDED_CUMULATIVE"] - cumulative_features2["o_REVERSALS_CUMULATIVE"]) / cumulative_features2["f_TAKEDOWNS_LANDED_CUMULATIVE"]
cumulative_features2["f_REVERSALS_ACCURACY/DEFENSE_MULT_CUMULATIVE"] = cumulative_features2["f_REVERSALS_ACCURACY_PROPORTION_CUMULATIVE"] * cumulative_features2["f_REVERSALS_DEFENSE_PROPORTION_CUMULATIVE"]
cumulative_features2["f_REVERSALS_PER_15_MINUTES_CUMULATIVE"] = cumulative_features2["f_REVERSALS_CUMULATIVE"] / cumulative_features2["f_TOTAL_TIME_MINUTES_CUMULATIVE"] * 15
cumulative_features2["f_REVERSALS_ABSORBED_PER_15_MINUTES_CUMULATIVE"] = cumulative_features2["o_REVERSALS_CUMULATIVE"] / cumulative_features2["f_TOTAL_TIME_MINUTES_CUMULATIVE"] * 15
cumulative_features2["f_REVERSALS/ABSORBED_PER_15_MINUTES_DIFF_CUMULATIVE"] = cumulative_features2["f_REVERSALS_PER_15_MINUTES_CUMULATIVE"] - cumulative_features2["f_REVERSALS_ABSORBED_PER_15_MINUTES_CUMULATIVE"]
cumulative_features2["f_REVERSALS/ABSORBED_DIFF_CUMULATIVE"] = cumulative_features2["f_REVERSALS_CUMULATIVE"] - cumulative_features2["o_REVERSALS_CUMULATIVE"]

cumulative_features2["f_CONTROL_TIME_PROPORTION_CUMULATIVE"] = cumulative_features2["f_CONTROL_TIME_MINUTES_CUMULATIVE"] / cumulative_features2["f_TOTAL_TIME_MINUTES_CUMULATIVE"]
cumulative_features2["f_CONTROL_TIME_GIVEN_UP_PROPORTION_CUMULATIVE"] = cumulative_features2["o_CONTROL_TIME_MINUTES_CUMULATIVE"] / cumulative_features2["f_TOTAL_TIME_MINUTES_CUMULATIVE"]
cumulative_features2["f_CONTROL_TIME/CONTROL_TIME_GIVEN_UP_PROPORTION_DIFF_CUMULATIVE"] = cumulative_features2["f_CONTROL_TIME_PROPORTION_CUMULATIVE"] - cumulative_features2["f_CONTROL_TIME_GIVEN_UP_PROPORTION_CUMULATIVE"]
cumulative_features2["f_CONTROL_TIME/CONTROL_TIME_GIVEN_UP_DIFF_CUMULATIVE"] = cumulative_features2["f_CONTROL_TIME_MINUTES_CUMULATIVE"] - cumulative_features2["o_CONTROL_TIME_MINUTES_CUMULATIVE"]

cumulative_features2 = cumulative_features2.copy()

cumulative_features2["f_SIGNIFICANT_STRIKES_ACCURACY_PROPORTION_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_LANDED_CUMULATIVE"] / cumulative_features2["f_SIGNIFICANT_STRIKES_ATTEMPTED_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_DEFENSE_PROPORTION_CUMULATIVE"] = (cumulative_features2["o_SIGNIFICANT_STRIKES_ATTEMPTED_CUMULATIVE"] - cumulative_features2["o_SIGNIFICANT_STRIKES_LANDED_CUMULATIVE"]) / cumulative_features2["o_SIGNIFICANT_STRIKES_ATTEMPTED_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_ACCURACY/DEFENSE_MULT_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_ACCURACY_PROPORTION_CUMULATIVE"] * cumulative_features2["f_SIGNIFICANT_STRIKES_DEFENSE_PROPORTION_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_LANDED_PER_MINUTE_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_LANDED_CUMULATIVE"] / cumulative_features2["f_TOTAL_TIME_MINUTES_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_ABSORBED_PER_MINUTE_CUMULATIVE"] = cumulative_features2["o_SIGNIFICANT_STRIKES_LANDED_CUMULATIVE"] / cumulative_features2["f_TOTAL_TIME_MINUTES_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_LANDED/ABSORBED_PER_MINUTE_DIFF_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_LANDED_PER_MINUTE_CUMULATIVE"] - cumulative_features2["f_SIGNIFICANT_STRIKES_ABSORBED_PER_MINUTE_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_LANDED/ABSORBED_DIFF_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_LANDED_CUMULATIVE"] - cumulative_features2["o_SIGNIFICANT_STRIKES_LANDED_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_LANDED_OUT_OF_TOTAL_PROPORTION_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_LANDED_CUMULATIVE"] / cumulative_features2["f_TOTAL_STRIKES_LANDED_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_ABSORBED_OUT_OF_TOTAL_PROPORTION_CUMULATIVE"] = cumulative_features2["o_SIGNIFICANT_STRIKES_LANDED_CUMULATIVE"] / cumulative_features2["o_TOTAL_STRIKES_LANDED_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_LANDED/ABSORBED_OUT_OF_TOTAL_PROPORTION_DIFF_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_LANDED_OUT_OF_TOTAL_PROPORTION_CUMULATIVE"] - cumulative_features2["f_SIGNIFICANT_STRIKES_ABSORBED_OUT_OF_TOTAL_PROPORTION_CUMULATIVE"]

cumulative_features2["f_SIGNIFICANT_STRIKES_HEAD_ACCURACY_PROPORTION_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_HEAD_LANDED_CUMULATIVE"] / cumulative_features2["f_SIGNIFICANT_STRIKES_HEAD_ATTEMPTED_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_HEAD_DEFENSE_PROPORTION_CUMULATIVE"] = (cumulative_features2["o_SIGNIFICANT_STRIKES_HEAD_ATTEMPTED_CUMULATIVE"] - cumulative_features2["o_SIGNIFICANT_STRIKES_HEAD_LANDED_CUMULATIVE"]) / cumulative_features2["o_SIGNIFICANT_STRIKES_HEAD_ATTEMPTED_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_HEAD_ACCURACY/DEFENSE_MULT_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_HEAD_ACCURACY_PROPORTION_CUMULATIVE"] * cumulative_features2["f_SIGNIFICANT_STRIKES_HEAD_DEFENSE_PROPORTION_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_HEAD_LANDED_PER_MINUTE_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_HEAD_LANDED_CUMULATIVE"] / cumulative_features2["f_TOTAL_TIME_MINUTES_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_HEAD_ABSORBED_PER_MINUTE_CUMULATIVE"] = cumulative_features2["o_SIGNIFICANT_STRIKES_HEAD_LANDED_CUMULATIVE"] / cumulative_features2["f_TOTAL_TIME_MINUTES_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_HEAD_LANDED/ABSORBED_PER_MINUTE_DIFF_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_HEAD_LANDED_PER_MINUTE_CUMULATIVE"] - cumulative_features2["f_SIGNIFICANT_STRIKES_HEAD_ABSORBED_PER_MINUTE_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_HEAD_LANDED/ABSORBED_DIFF_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_HEAD_LANDED_CUMULATIVE"] - cumulative_features2["o_SIGNIFICANT_STRIKES_HEAD_LANDED_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_HEAD_LANDED_OUT_OF_SIGNIFICANT_PROPORTION_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_HEAD_LANDED_CUMULATIVE"] / cumulative_features2["f_SIGNIFICANT_STRIKES_LANDED_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_HEAD_ABSORBED_OUT_OF_SIGNIFICANT_PROPORTION_CUMULATIVE"] = cumulative_features2["o_SIGNIFICANT_STRIKES_HEAD_LANDED_CUMULATIVE"] / cumulative_features2["o_SIGNIFICANT_STRIKES_LANDED_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_HEAD_LANDED/ABSORBED_OUT_OF_SIGNIFICANT_PROPORTION_DIFF_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_HEAD_LANDED_OUT_OF_SIGNIFICANT_PROPORTION_CUMULATIVE"] - cumulative_features2["f_SIGNIFICANT_STRIKES_HEAD_ABSORBED_OUT_OF_SIGNIFICANT_PROPORTION_CUMULATIVE"]

cumulative_features2["f_SIGNIFICANT_STRIKES_BODY_ACCURACY_PROPORTION_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_BODY_LANDED_CUMULATIVE"] / cumulative_features2["f_SIGNIFICANT_STRIKES_BODY_ATTEMPTED_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_BODY_DEFENSE_PROPORTION_CUMULATIVE"] = (cumulative_features2["o_SIGNIFICANT_STRIKES_BODY_ATTEMPTED_CUMULATIVE"] - cumulative_features2["o_SIGNIFICANT_STRIKES_BODY_LANDED_CUMULATIVE"]) / cumulative_features2["o_SIGNIFICANT_STRIKES_BODY_ATTEMPTED_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_BODY_ACCURACY/DEFENSE_MULT_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_BODY_ACCURACY_PROPORTION_CUMULATIVE"] * cumulative_features2["f_SIGNIFICANT_STRIKES_BODY_DEFENSE_PROPORTION_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_BODY_LANDED_PER_MINUTE_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_BODY_LANDED_CUMULATIVE"] / cumulative_features2["f_TOTAL_TIME_MINUTES_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_BODY_ABSORBED_PER_MINUTE_CUMULATIVE"] = cumulative_features2["o_SIGNIFICANT_STRIKES_BODY_LANDED_CUMULATIVE"] / cumulative_features2["f_TOTAL_TIME_MINUTES_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_BODY_LANDED/ABSORBED_PER_MINUTE_DIFF_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_BODY_LANDED_PER_MINUTE_CUMULATIVE"] - cumulative_features2["f_SIGNIFICANT_STRIKES_BODY_ABSORBED_PER_MINUTE_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_BODY_LANDED/ABSORBED_DIFF_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_BODY_LANDED_CUMULATIVE"] - cumulative_features2["o_SIGNIFICANT_STRIKES_BODY_LANDED_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_BODY_LANDED_OUT_OF_SIGNIFICANT_PROPORTION_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_BODY_LANDED_CUMULATIVE"] / cumulative_features2["f_SIGNIFICANT_STRIKES_LANDED_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_BODY_ABSORBED_OUT_OF_SIGNIFICANT_PROPORTION_CUMULATIVE"] = cumulative_features2["o_SIGNIFICANT_STRIKES_BODY_LANDED_CUMULATIVE"] / cumulative_features2["o_SIGNIFICANT_STRIKES_LANDED_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_BODY_LANDED/ABSORBED_OUT_OF_SIGNIFICANT_PROPORTION_DIFF_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_BODY_LANDED_OUT_OF_SIGNIFICANT_PROPORTION_CUMULATIVE"] - cumulative_features2["f_SIGNIFICANT_STRIKES_BODY_ABSORBED_OUT_OF_SIGNIFICANT_PROPORTION_CUMULATIVE"]

cumulative_features2["f_SIGNIFICANT_STRIKES_LEG_ACCURACY_PROPORTION_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_LEG_LANDED_CUMULATIVE"] / cumulative_features2["f_SIGNIFICANT_STRIKES_LEG_ATTEMPTED_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_LEG_DEFENSE_PROPORTION_CUMULATIVE"] = (cumulative_features2["o_SIGNIFICANT_STRIKES_LEG_ATTEMPTED_CUMULATIVE"] - cumulative_features2["o_SIGNIFICANT_STRIKES_LEG_LANDED_CUMULATIVE"]) / cumulative_features2["o_SIGNIFICANT_STRIKES_LEG_ATTEMPTED_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_LEG_ACCURACY/DEFENSE_MULT_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_LEG_ACCURACY_PROPORTION_CUMULATIVE"] * cumulative_features2["f_SIGNIFICANT_STRIKES_LEG_DEFENSE_PROPORTION_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_LEG_LANDED_PER_MINUTE_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_LEG_LANDED_CUMULATIVE"] / cumulative_features2["f_TOTAL_TIME_MINUTES_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_LEG_ABSORBED_PER_MINUTE_CUMULATIVE"] = cumulative_features2["o_SIGNIFICANT_STRIKES_LEG_LANDED_CUMULATIVE"] / cumulative_features2["f_TOTAL_TIME_MINUTES_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_LEG_LANDED/ABSORBED_PER_MINUTE_DIFF_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_LEG_LANDED_PER_MINUTE_CUMULATIVE"] - cumulative_features2["f_SIGNIFICANT_STRIKES_LEG_ABSORBED_PER_MINUTE_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_LEG_LANDED/ABSORBED_DIFF_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_LEG_LANDED_CUMULATIVE"] - cumulative_features2["o_SIGNIFICANT_STRIKES_LEG_LANDED_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_LEG_LANDED_OUT_OF_SIGNIFICANT_PROPORTION_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_LEG_LANDED_CUMULATIVE"] / cumulative_features2["f_SIGNIFICANT_STRIKES_LANDED_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_LEG_ABSORBED_OUT_OF_SIGNIFICANT_PROPORTION_CUMULATIVE"] = cumulative_features2["o_SIGNIFICANT_STRIKES_LEG_LANDED_CUMULATIVE"] / cumulative_features2["o_SIGNIFICANT_STRIKES_LANDED_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_LEG_LANDED/ABSORBED_OUT_OF_SIGNIFICANT_PROPORTION_DIFF_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_LEG_LANDED_OUT_OF_SIGNIFICANT_PROPORTION_CUMULATIVE"] - cumulative_features2["f_SIGNIFICANT_STRIKES_LEG_ABSORBED_OUT_OF_SIGNIFICANT_PROPORTION_CUMULATIVE"]

cumulative_features2["f_SIGNIFICANT_STRIKES_DISTANCE_ACCURACY_PROPORTION_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_DISTANCE_LANDED_CUMULATIVE"] / cumulative_features2["f_SIGNIFICANT_STRIKES_DISTANCE_ATTEMPTED_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_DISTANCE_DEFENSE_PROPORTION_CUMULATIVE"] = (cumulative_features2["o_SIGNIFICANT_STRIKES_DISTANCE_ATTEMPTED_CUMULATIVE"] - cumulative_features2["o_SIGNIFICANT_STRIKES_DISTANCE_LANDED_CUMULATIVE"]) / cumulative_features2["o_SIGNIFICANT_STRIKES_DISTANCE_ATTEMPTED_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_DISTANCE_ACCURACY/DEFENSE_MULT_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_DISTANCE_ACCURACY_PROPORTION_CUMULATIVE"] * cumulative_features2["f_SIGNIFICANT_STRIKES_DISTANCE_DEFENSE_PROPORTION_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_DISTANCE_LANDED_PER_MINUTE_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_DISTANCE_LANDED_CUMULATIVE"] / cumulative_features2["f_TOTAL_TIME_MINUTES_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_DISTANCE_ABSORBED_PER_MINUTE_CUMULATIVE"] = cumulative_features2["o_SIGNIFICANT_STRIKES_DISTANCE_LANDED_CUMULATIVE"] / cumulative_features2["f_TOTAL_TIME_MINUTES_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_DISTANCE_LANDED/ABSORBED_PER_MINUTE_DIFF_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_DISTANCE_LANDED_PER_MINUTE_CUMULATIVE"] - cumulative_features2["f_SIGNIFICANT_STRIKES_DISTANCE_ABSORBED_PER_MINUTE_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_DISTANCE_LANDED/ABSORBED_DIFF_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_DISTANCE_LANDED_CUMULATIVE"] - cumulative_features2["o_SIGNIFICANT_STRIKES_DISTANCE_LANDED_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_DISTANCE_LANDED_OUT_OF_SIGNIFICANT_PROPORTION_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_DISTANCE_LANDED_CUMULATIVE"] / cumulative_features2["f_SIGNIFICANT_STRIKES_LANDED_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_DISTANCE_ABSORBED_OUT_OF_SIGNIFICANT_PROPORTION_CUMULATIVE"] = cumulative_features2["o_SIGNIFICANT_STRIKES_DISTANCE_LANDED_CUMULATIVE"] / cumulative_features2["o_SIGNIFICANT_STRIKES_LANDED_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_DISTANCE_LANDED/ABSORBED_OUT_OF_SIGNIFICANT_PROPORTION_DIFF_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_DISTANCE_LANDED_OUT_OF_SIGNIFICANT_PROPORTION_CUMULATIVE"] - cumulative_features2["f_SIGNIFICANT_STRIKES_DISTANCE_ABSORBED_OUT_OF_SIGNIFICANT_PROPORTION_CUMULATIVE"]

cumulative_features2["f_SIGNIFICANT_STRIKES_CLINCH_ACCURACY_PROPORTION_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_CLINCH_LANDED_CUMULATIVE"] / cumulative_features2["f_SIGNIFICANT_STRIKES_CLINCH_ATTEMPTED_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_CLINCH_DEFENSE_PROPORTION_CUMULATIVE"] = (cumulative_features2["o_SIGNIFICANT_STRIKES_CLINCH_ATTEMPTED_CUMULATIVE"] - cumulative_features2["o_SIGNIFICANT_STRIKES_CLINCH_LANDED_CUMULATIVE"]) / cumulative_features2["o_SIGNIFICANT_STRIKES_CLINCH_ATTEMPTED_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_CLINCH_ACCURACY/DEFENSE_MULT_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_CLINCH_ACCURACY_PROPORTION_CUMULATIVE"] * cumulative_features2["f_SIGNIFICANT_STRIKES_CLINCH_DEFENSE_PROPORTION_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_CLINCH_LANDED_PER_MINUTE_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_CLINCH_LANDED_CUMULATIVE"] / cumulative_features2["f_TOTAL_TIME_MINUTES_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_CLINCH_ABSORBED_PER_MINUTE_CUMULATIVE"] = cumulative_features2["o_SIGNIFICANT_STRIKES_CLINCH_LANDED_CUMULATIVE"] / cumulative_features2["f_TOTAL_TIME_MINUTES_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_CLINCH_LANDED/ABSORBED_PER_MINUTE_DIFF_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_CLINCH_LANDED_PER_MINUTE_CUMULATIVE"] - cumulative_features2["f_SIGNIFICANT_STRIKES_CLINCH_ABSORBED_PER_MINUTE_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_CLINCH_LANDED/ABSORBED_DIFF_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_CLINCH_LANDED_CUMULATIVE"] - cumulative_features2["o_SIGNIFICANT_STRIKES_CLINCH_LANDED_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_CLINCH_LANDED_OUT_OF_SIGNIFICANT_PROPORTION_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_CLINCH_LANDED_CUMULATIVE"] / cumulative_features2["f_SIGNIFICANT_STRIKES_LANDED_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_CLINCH_ABSORBED_OUT_OF_SIGNIFICANT_PROPORTION_CUMULATIVE"] = cumulative_features2["o_SIGNIFICANT_STRIKES_CLINCH_LANDED_CUMULATIVE"] / cumulative_features2["o_SIGNIFICANT_STRIKES_LANDED_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_CLINCH_LANDED/ABSORBED_OUT_OF_SIGNIFICANT_PROPORTION_DIFF_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_CLINCH_LANDED_OUT_OF_SIGNIFICANT_PROPORTION_CUMULATIVE"] - cumulative_features2["f_SIGNIFICANT_STRIKES_CLINCH_ABSORBED_OUT_OF_SIGNIFICANT_PROPORTION_CUMULATIVE"]

cumulative_features2["f_SIGNIFICANT_STRIKES_GROUND_ACCURACY_PROPORTION_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_GROUND_LANDED_CUMULATIVE"] / cumulative_features2["f_SIGNIFICANT_STRIKES_GROUND_ATTEMPTED_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_GROUND_DEFENSE_PROPORTION_CUMULATIVE"] = (cumulative_features2["o_SIGNIFICANT_STRIKES_GROUND_ATTEMPTED_CUMULATIVE"] - cumulative_features2["o_SIGNIFICANT_STRIKES_GROUND_LANDED_CUMULATIVE"]) / cumulative_features2["o_SIGNIFICANT_STRIKES_GROUND_ATTEMPTED_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_GROUND_ACCURACY/DEFENSE_MULT_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_GROUND_ACCURACY_PROPORTION_CUMULATIVE"] * cumulative_features2["f_SIGNIFICANT_STRIKES_GROUND_DEFENSE_PROPORTION_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_GROUND_LANDED_PER_MINUTE_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_GROUND_LANDED_CUMULATIVE"] / cumulative_features2["f_TOTAL_TIME_MINUTES_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_GROUND_ABSORBED_PER_MINUTE_CUMULATIVE"] = cumulative_features2["o_SIGNIFICANT_STRIKES_GROUND_LANDED_CUMULATIVE"] / cumulative_features2["f_TOTAL_TIME_MINUTES_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_GROUND_LANDED/ABSORBED_PER_MINUTE_DIFF_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_GROUND_LANDED_PER_MINUTE_CUMULATIVE"] - cumulative_features2["f_SIGNIFICANT_STRIKES_GROUND_ABSORBED_PER_MINUTE_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_GROUND_LANDED/ABSORBED_DIFF_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_GROUND_LANDED_CUMULATIVE"] - cumulative_features2["o_SIGNIFICANT_STRIKES_GROUND_LANDED_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_GROUND_LANDED_OUT_OF_SIGNIFICANT_PROPORTION_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_GROUND_LANDED_CUMULATIVE"] / cumulative_features2["f_SIGNIFICANT_STRIKES_LANDED_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_GROUND_ABSORBED_OUT_OF_SIGNIFICANT_PROPORTION_CUMULATIVE"] = cumulative_features2["o_SIGNIFICANT_STRIKES_GROUND_LANDED_CUMULATIVE"] / cumulative_features2["o_SIGNIFICANT_STRIKES_LANDED_CUMULATIVE"]
cumulative_features2["f_SIGNIFICANT_STRIKES_GROUND_LANDED/ABSORBED_OUT_OF_SIGNIFICANT_PROPORTION_DIFF_CUMULATIVE"] = cumulative_features2["f_SIGNIFICANT_STRIKES_GROUND_LANDED_OUT_OF_SIGNIFICANT_PROPORTION_CUMULATIVE"] - cumulative_features2["f_SIGNIFICANT_STRIKES_GROUND_ABSORBED_OUT_OF_SIGNIFICANT_PROPORTION_CUMULATIVE"]

cumulative_features2

,f_TOTAL_TIME_MINUTES_CUMULATIVE,f_KNOCKDOWNS_CUMULATIVE,f_TOTAL_STRIKES_LANDED_CUMULATIVE,f_TOTAL_STRIKES_ATTEMPTED_CUMULATIVE,f_TAKEDOWNS_LANDED_CUMULATIVE,f_TAKEDOWNS_ATTEMPTED_CUMULATIVE,f_SUBMISSION_ATTEMPTS_CUMULATIVE,f_REVERSALS_CUMULATIVE,f_CONTROL_TIME_MINUTES_CUMULATIVE,f_SIGNIFICANT_STRIKES_LANDED_CUMULATIVE,f_SIGNIFICANT_STRIKES_ATTEMPTED_CUMULATIVE,f_SIGNIFICANT_STRIKES_HEAD_LANDED_CUMULATIVE,f_SIGNIFICANT_STRIKES_HEAD_ATTEMPTED_CUMULATIVE,f_SIGNIFICANT_STRIKES_BODY_LANDED_CUMULATIVE,f_SIGNIFICANT_STRIKES_BODY_ATTEMPTED_CUMULATIVE,f_SIGNIFICANT_STRIKES_LEG_LANDED_CUMULATIVE,f_SIGNIFICANT_STRIKES_LEG_ATTEMPTED_CUMULATIVE,f_SIGNIFICANT_STRIKES_DISTANCE_LANDED_CUMULATIVE,f_SIGNIFICANT_STRIKES_DISTANCE_ATTEMPTED_CUMULATIVE,f_SIGNIFICANT_STRIKES_CLINCH_LANDED_CUMULATIVE,f_SIGNIFICANT_STRIKES_CLINCH_ATTEMPTED_CUMULATIVE,f_SIGNIFICANT_STRIKES_GROUND_LANDED_CUMULATIVE,f_SIGNIFICANT_STRIKES_GROUND_ATTEMPTED_CUMULATIVE,o_KNOCKDOWNS_CUMULATIVE,o_TOTAL_STRIKES_LANDED_CUMULATIVE,o_TOTAL_STRIKES_ATTEMPTED_CUMULATIVE,o_TAKEDOWNS_LANDED_CUMULATIVE,o_TAKEDOWNS_ATTEMPTED_CUMULATIVE,o_SUBMISSION_ATTEMPTS_CUMULATIVE,o_REVERSALS_CUMULATIVE,o_CONTROL_TIME_MINUTES_CUMULATIVE,o_SIGNIFICANT_STRIKES_LANDED_CUMULATIVE,o_SIGNIFICANT_STRIKES_ATTEMPTED_CUMULATIVE,o_SIGNIFICANT_STRIKES_HEAD_LANDED_CUMULATIVE,o_SIGNIFICANT_STRIKES_HEAD_ATTEMPTED_CUMULATIVE,o_SIGNIFICANT_STRIKES_BODY_LANDED_CUMULATIVE,o_SIGNIFICANT_STRIKES_BODY_ATTEMPTED_CUMULATIVE,o_SIGNIFICANT_STRIKES_LEG_LANDED_CUMULATIVE,o_SIGNIFICANT_STRIKES_LEG_ATTEMPTED_CUMULATIVE,o_SIGNIFICANT_STRIKES_DISTANCE_LANDED_CUMULATIVE,o_SIGNIFICANT_STRIKES_DISTANCE_ATTEMPTED_CUMULATIVE,o_SIGNIFICANT_STRIKES_CLINCH_LANDED_CUMULATIVE,o_SIGNIFICANT_STRIKES_CLINCH_ATTEMPTED_CUMULATIVE,o_SIGNIFICANT_STRIKES_GROUND_LANDED_CUMULATIVE,o_SIGNIFICANT_STRIKES_GROUND_ATTEMPTED_CUMULATIVE,f_TITLE_BOUTS_FOUGHT,f_UNDISPUTED_TITLE_BOUTS_FOUGHT,f_FIVE_ROUND_BOUTS_FOUGHT,f_WINS_CUMULATIVE,f_LOSSES_CUMULATIVE,f_DRAWS_CUMULATIVE,f_NO_CONTESTS_CUMULATIVE,f_TOTAL_BOUTS_FOUGHT,f_WINS_BY_KO/TKO_CUMULATIVE,f_WINS_BY_SUBMISSION_CUMULATIVE,f_WINS_BY_DECISION_CUMULATIVE,f_WINS_FIRST_ROUND_FINISH_CUMULATIVE,f_LOSSES_BY_KO/TKO_CUMULATIVE,f_LOSSES_BY_SUBMISSION_CUMULATIVE,f_LOSSES_BY_DECISION_CUMULATIVE,f_KNOCKDOWNS_LANDED_PER_15_MINUTES_CUMULATIVE,f_KNOCKDOWNS_ABSORBED_PER_15_MINUTES_CUMULATIVE,f_KNOCKDOWNS_LANDED/ABSORBED_PER_15_MINUTES_DIFF_CUMULATIVE,f_KNOCKDOWNS_LANDED/ABSORBED_DIFF_CUMULATIVE,f_TOTAL_STRIKES_ACCURACY_PROPORTION_CUMULATIVE,f_TOTAL_STRIKES_DEFENSE_PROPORTION_CUMULATIVE,f_TOTAL_STRIKES_ACCURACY/DEFENSE_MULT_CUMULATIVE,f_TOTAL_STRIKES_LANDED_PER_MINUTE_CUMULATIVE,f_TOTAL_STRIKES_ABSORBED_PER_MINUTE_CUMULATIVE,f_TOTAL_STRIKES_LANDED/ABSORBED_PER_MINUTE_DIFF_CUMULATIVE,f_TOTAL_STRIKES_LANDED/ABSORBED_DIFF_CUMULATIVE,f_TAKEDOWNS_ACCURACY_PROPORTION_CUMULATIVE,f_TAKEDOWNS_DEFENSE_PROPORTION_CUMULATIVE,f_TAKEDOWNS_ACCURACY/DEFENSE_MULT_CUMULATIVE,f_TAKEDOWNS_LANDED_PER_15_MINUTES_CUMULATIVE,f_TAKEDOWNS_ABSORBED_PER_15_MINUTES_CUMULATIVE,f_TAKEDOWNS_LANDED/ABSORBED_PER_15_MINUTES_DIFF_CUMULATIVE,f_TAKEDOWNS_LANDED/ABSORBED_DIFF_CUMULATIVE,f_SUBMISSION_ATTEMPTS_ACCURACY_PROPORTION_CUMULATIVE,f_SUBMISSION_ATTEMPTS_DEFENSE_PROPORTION_CUMULATIVE,f_SUBMISSION_ATTEMPTS_ACCURACY/DEFENSE_MULT_CUMULATIVE,f_SUBMISSION_ATTEMPTS_PER_15_MINUTES_CUMULATIVE,f_SUBMISSION_ATTEMPTS_ABSORBED_PER_15_MINUTES_CUMULATIVE,f_SUBMISSION_ATTEMPTS/ABSORBED_PER_15_MINUTES_DIFF_CUMULATIVE,f_SUBMISSION_ATTEMPTS/ABSORBED_DIFF_CUMULATIVE,f_REVERSALS_ACCURACY_PROPORTION_CUMULATIVE,f_REVERSALS_DEFENSE_PROPORTION_CUMULATIVE,f_REVERSALS_ACCURACY/DEFENSE_MULT_CUMULATIVE,f_REVERSALS_PER_15_MINUTES_CUMULATIVE,f_REVERSALS_ABSORBED_PER_15_MINUTES_CUMULATIVE,f_REVERSALS/ABSORBED_PER_15_MINUTES_DIFF_CUMULATIVE,f_REVERSALS/ABSORBED_DIFF_CUMULATIVE,f_CONTROL_TIME_PROPORTION_CUMULATIVE,f_CONTROL_TIME_GIVEN_UP_PROPORTION_CUMULATIVE,f_CONTROL_TIME/CONTROL_TIME_GIVEN_UP_PROPORTION_DIFF_CUMULATIVE,f_CONTROL_TIME/CONTROL_TIME_GIVEN_UP_DIFF_CUMULATIVE,f_SIGNIFICANT

In [ ]:
cumulative_features3 = cumulative_features2.copy()


In [ ]:
# cumulative
# rolling 3 fight window
# lag 1 fight

# accuracy: f_landed / f_attempted
# avg landed per 15 min: f_landed / total_time * 15
# defense: (o_attempted - o_landed) / o_attempted
# avg absorbed per 15 min: o_landed / total_time * 15

# win pct (count draws as 0.5)
# winning and losing streaks
# finish pct
# wins by ko/tko, sub, dec

# avg landed per 15 min / avg absorbed per 15 min
# accuracy * defense
# total landed - total absorbed
# ground_strikes / control_time * 15
# (ground_strikes / control_time) / (takedowns_landed + reversals)
# control_time / total_time
# control_time / (takedowns_landed + reversals)
# (ko/tko_wins + knockdowns) / significant strikes landed
# head / total sig, body / total sig, leg / total sig, distance / total sig, clinch / total sig, ground / total sig
# number of days since last fight
# cte score: age * head_absorbed * (1 + ko/tko_losses)

# custom elo rating system
# elo score, elo probability

# mins and maxes
# deltas in stats between bouts

# title fight indicator
# champ indicator
# number of title fights
# number of 5-round fights participated in
# title wins

# finally finally ratios/differences between stats between fighters

# at the end, drop all number of attempted columns

### Coalesce the features into a single dataframe that will serve as input

In [12]:
data = pd.DataFrame()
# data["BOUT_ID"] = df.loc[
#     (pd.to_datetime(df["DATE"]) >= pd.to_datetime("2010-03-21")) & (df["RED_OUTCOME"].isin(["W", "L"])),
#     "BOUT_ID",
# ]
data = df.loc[
    (pd.to_datetime(df["DATE"]) >= pd.to_datetime("2013-02-04")) & (df["RED_OUTCOME"].isin(["W", "L"]))
]

# Map back features

# Reset index
data.reset_index(inplace=True, drop=True)
# data.loc[(data["RED_DATE_OF_BIRTH"].isnull()) | (data["BLUE_DATE_OF_BIRTH"].isnull())]
data

,BOUT_ID,EVENT_ID,EVENT_NAME,DATE,LOCATION,RED_FIGHTER_ID,BLUE_FIGHTER_ID,RED_FIGHTER_NAME,BLUE_FIGHTER_NAME,RED_OUTCOME,BLUE_OUTCOME,BOUT_TYPE,OUTCOME_METHOD,OUTCOME_METHOD_DETAILS,END_ROUND,END_ROUND_TIME_MINUTES,BOUT_TIME_FORMAT,TOTAL_TIME_MINUTES,RED_KNOCKDOWNS,BLUE_KNOCKDOWNS,RED_TOTAL_STRIKES_LANDED,RED_TOTAL_STRIKES_ATTEMPTED,BLUE_TOTAL_STRIKES_LANDED,BLUE_TOTAL_STRIKES_ATTEMPTED,RED_TAKEDOWNS_LANDED,RED_TAKEDOWNS_ATTEMPTED,BLUE_TAKEDOWNS_LANDED,BLUE_TAKEDOWNS_ATTEMPTED,RED_SUBMISSION_ATTEMPTS,BLUE_SUBMISSION_ATTEMPTS,RED_REVERSALS,BLUE_REVERSALS,RED_CONTROL_TIME_MINUTES,BLUE_CONTROL_TIME_MINUTES,RED_SIGNIFICANT_STRIKES_LANDED,RED_SIGNIFICANT_STRIKES_ATTEMPTED,BLUE_SIGNIFICANT_STRIKES_LANDED,BLUE_SIGNIFICANT_STRIKES_ATTEMPTED,RED_SIGNIFICANT_STRIKES_HEAD_LANDED,RED_SIGNIFICANT_STRIKES_HEAD_ATTEMPTED,BLUE_SIGNIFICANT_STRIKES_HEAD_LANDED,BLUE_SIGNIFICANT_STRIKES_HEAD_ATTEMPTED,RED_SIGNIFICANT_STRIKES_BODY_LANDED,RED_SIGNIFICANT_STRIKES_BODY_ATTEMPTED,BLUE_SIGNIFICANT_STRIKES_BODY_LANDED,BLUE_SIGNIFICANT_STRIKES_BODY_ATTEMPTED,RED_SIGNIFICANT_STRIKES_LEG_LANDED,RED_SIGNIFICANT_STRIKES_LEG_ATTEMPTED,BLUE_SIGNIFICANT_STRIKES_LEG_LANDED,BLUE_SIGNIFICANT_STRIKES_LEG_ATTEMPTED,RED_SIGNIFICANT_STRIKES_DISTANCE_LANDED,RED_SIGNIFICANT_STRIKES_DISTANCE_ATTEMPTED,BLUE_SIGNIFICANT_STRIKES_DISTANCE_LANDED,BLUE_SIGNIFICANT_STRIKES_DISTANCE_ATTEMPTED,RED_SIGNIFICANT_STRIKES_CLINCH_LANDED,RED_SIGNIFICANT_STRIKES_CLINCH_ATTEMPTED,BLUE_SIGNIFICANT_STRIKES_CLINCH_LANDED,BLUE_SIGNIFICANT_STRIKES_CLINCH_ATTEMPTED,RED_SIGNIFICANT_STRIKES_GROUND_LANDED,RED_SIGNIFICANT_STRIKES_GROUND_ATTEMPTED,BLUE_SIGNIFICANT_STRIKES_GROUND_LANDED,BLUE_SIGNIFICANT_STRIKES_GROUND_ATTEMPTED,RED_HEIGHT_INCHES,RED_REACH_INCHES,RED_FIGHTING_STANCE,RED_DATE_OF_BIRTH,BLUE_HEIGHT_INCHES,BLUE_REACH_INCHES,BLUE_FIGHTING_STANCE,BLUE_DATE_OF_BIRTH
0,16ba92318ff66d4a,4679a38cced7c64a,UFC on FUEL TV: Barao vs McDonald,2013-02-16,"London, England, United Kingdom",ba043b03eb799063,1e35aa8955794580,Phil Harris,Ulysses Gomez,W,L,Flyweight Bout,Decision - Unanimous,Aaron Chatfield \n\n \n ...,3,5.000000,3 Rnd (5-5-5),15.000000,0.0,0.0,33.0,150.0,35.0,159.0,1.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,0.150000,0.133333,33.0,150.0,35.0,159.0,12.0,121.0,8.0,111.0,3.0,8.0,0.0,4.0,18.0,21.0,27.0,44.0,29.0,143.0,35.0,159.0,4.0,5.0,0.0,0.0,0.0,2.0,0.0,0.0,64.0,65.0,Orthodox,1983-09-04,66.0,NaN,Orthodox,1983-05-25
1,f509d5ffcc3f46fe,4679a38cced7c64a,UFC on FUEL TV: Barao vs McDonald,2013-02-16,"London, England, United Kingdom",8bb6f4479685bbcc,d34636a85f0f4c90,Vaughan Lee,Motonobu Tezuka,W,L,Bantamweight Bout,Decision - Unanimous,Tom Harris \n\n \n \n ...,3,5.000000,3 Rnd (5-5-5),15.000000,0.0,0.0,164.0,192.0,17.0,56.0,0.0,0.0,1.0,15.0,0.0,1.0,0.0,0.0,1.183333,7.850000,20.0,48.0,4.0,35.0,7.0,28.0,1.0,28.0,9.0,14.0,0.0,0.0,4.0,6.0,3.0,7.0,6.0,28.0,4.0,34.0,12.0,18.0,0.0,1.0,2.0,2.0,0.0,0.0,66.0,66.0,Orthodox,1982-10-15,67.0,NaN,Southpaw,1987-08-29
2,22a574162e2ebcf6,4679a38cced7c64a,UFC on FUEL TV: Barao vs McDonald,2013-02-16,"London, England, United Kingdom",6809c31286b08d97,1d4bbaabd8b60173,Tom Watson,Stanislav Nedkov,W,L,Middleweight Bout,KO/TKO,Knee to Body In Clinch,2,4.700000,3 Rnd (5-5-5),9.700000,1.0,0.0,72.0,118.0,61.0,102.0,0.0,0.0,5.0,6.0,0.0,0.0,0.0,0.0,4.400000,2.666667,56.0,102.0,39.0,78.0,31.0,66.0,26.0,62.0,23.0,34.0,13.0,16.0,2.0,2.0,0.0,0.0,3.0,16.0,5.0,18.0,50.0,81.0,19.0,29.0,3.0,5.0,15.0,31.0,73.0,73.0,Orthodox,1982-07-13,71.0,71.0,Orthodox,1981-12-09
3,d607908be8b19273,4679a38cced7c64a,UFC on FUEL TV: Barao vs McDonald,2013-02-16,"London, England, United Kingdom",b6c4451cb13c9303,60ba33c3f555a7a3,Andy Ogle,Josh Grispi,W,L,Featherweight Bout,Decision - Unanimous,Chris Lee \n\n \n \n ...,3,5.000000,3 Rnd (5-5-5),15.000000,0.0,0.0,119.0,153.0,19.0,38.0,2.0,2.0,1.0,2.0,0.0,2.0,1.0,0.0,13.183333,0.316667,19.0,42.0,7.0,26.0,10.0,32.0,0.0,15.0,8.0,9.0,2.0,4.0,1.0,1.0,5.0,7.0,8.0,29.0,6.0,25.0,0.0,0.0,1.0,1.0,11.0,13.0,0.0,0.0,68.0,69.0,Orthodox,1989-02-16,71.0,74.0,Orthodox,1988-10-14


In [ ]:
# Split into train and test sets
split_index = int(len(data) * 0.8)
train = data[:split_index]
test = data[split_index:]

X_train, y_train = train.drop("TARGET", axis=1), train["TARGET"]
X_test, y_test = test.drop("TARGET", axis=1), test["TARGET"]
len(test)

## AutoML Benchmarking

In [ ]:
import evalml
from evalml.preprocessing.data_splitters import TimeSeriesSplit

In [ ]:
automl = evalml.AutoMLSearch(
    X_train=X_train,
    y_train=y_train,
    X_holdout=X_test,
    y_holdout=y_test,
    problem_type="binary",
    objective="log loss binary",
    data_splitter=TimeSeriesSplit(n_splits=5),
)